In [1]:
%pip install langchain google-generativeai
%pip install faiss-cpu lancedb tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["GOOGLE_API_KEY"] = "add_your_api"

In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:

llm = GoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
)

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage
messages =[
    
        SystemMessage(content="You are a helpful assistant that translates English to French. Translate the user sentence."),
        HumanMessage(content="Hi, My name is Rishav Singh. What's your name?"),
        ]

In [14]:
ai_msg = llm.invoke(messages)
ai_msg

"Bonjour, je m'appelle Rishav Singh. Comment vous appelez-vous ?"

In [15]:
from langchain.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("/Users/rishavsingh/Desktop/albert camus.docx")
documents = loader.load()
len(documents)

1

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_plitter=RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
split=text_plitter.split_documents(documents)
print(len(split))                   #5 splits      
print(split[1].page_content)        #testing the text splitter

5
In the post-war period, Camus became a prominent public intellectual, known for his moral clarity and opposition to totalitarianism, including his criticism of both fascism and communism. He won the Nobel Prize in Literature in 1957 for illuminating the human conscience in the face of absurdity. His later works, including The Plague (1947) and The Fall(1956), explored themes of suffering, justice, and human solidarity. Tragically, Camus died in a car accident on January 4, 1960, at the age of 46, cutting short a career that had significantly influenced modern thought. His legacy continues to shape discussions on morality, freedom, and the human condition.


In [32]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/y5/p42j4t91737_2v19wfkt3sv80000gn/T/ipykernel_29171/2848935613.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [33]:
from langchain.vectorstores import FAISS
vectorDB=FAISS.from_documents(split, embedding)

In [52]:
#Way 1 to test the query with similary search of Faiss
query = "Why was Albert Camus interested in the absurd?"
answer=vectorDB.similarity_search(query,k=2)

In [54]:
print(answer[1].page_content) #testing the vectorDB

Unlike existentialists like Jean-Paul Sartre, Camus did not believe that we could create our own meaning through radical freedom or choice. Instead, he advocated for lucid rebellion—a life lived with awareness, integrity, and defiance against the absurd. His later works, especially The Plague, reflect his belief in human solidarity, where moral action and empathy become necessary responses to a world without divine justice. For Camus, the point of life wasn't to find meaning, but to live honestly, with compassion and dignity, in the face of meaninglessness.

Here are some key quotes by Albert Camus that capture the essence of his philosophy:

On the Absurd:
"There is only one really serious philosophical problem, and that is suicide."
— The Myth of Sisyphus
(Camus opens with this to emphasize how confronting life's meaninglessness is the central philosophical question.)


In [55]:
from langchain.chains import RetrievalQA

In [57]:
qna=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorDB.as_retriever()
)

In [58]:
query = "Why was Albert Camus interested in the absurd?"
response=qna.run(query)
print(response)

/var/folders/y5/p42j4t91737_2v19wfkt3sv80000gn/T/ipykernel_29171/1156458997.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=qna.run(query)


Albert Camus was interested in the absurd because it is the conflict between humans’ search for meaning and the indifferent, meaningless universe. He saw confronting life's meaninglessness as the central philosophical question.
